# Templete

In [133]:
!pip install catboost

# Library & Config

## Library

In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import random
import os
from tqdm import tqdm

# scipy
from scipy.stats import zscore

# PreProcessing
## imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

## encoder
from sklearn.preprocessing import LabelEncoder

## scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

## model selection
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    RandomizedSearchCV,
    cross_val_score,
    cross_val_predict,
    KFold
)

# ML model
## Ensemble Methods
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier
)

## GLM
from sklearn.linear_model import (
    LogisticRegression,
    PassiveAggressiveClassifier,
)

## Trees
from sklearn.tree import (
    DecisionTreeClassifier,
    ExtraTreeClassifier
)

## Naive Bayes
from sklearn.naive_bayes import GaussianNB

## K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

## Support Vector Machine
from sklearn.svm import SVC

## Boosting Methods
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Scoring
from sklearn.metrics import accuracy_score, f1_score
from sklearn.base import clone

# PyTorch
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import random
import os
from tqdm import tqdm

# scipy
from scipy.stats import zscore

# PreProcessing
## imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

## encoder
from sklearn.preprocessing import LabelEncoder

## scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

## model selection
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    RandomizedSearchCV,
    cross_val_score,
    cross_val_predict,
    KFold
)

# ML model
## Ensemble Methods
from sklearn.ensemble import (
    AdaBoostClassifier,
    BaggingClassifier,
    ExtraTreesClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier
)

## GLM
from sklearn.linear_model import (
    LogisticRegression,
    PassiveAggressiveClassifier,
)

## Trees
from sklearn.tree import (
    DecisionTreeClassifier,
    ExtraTreeClassifier
)

## Naive Bayes
from sklearn.naive_bayes import GaussianNB

## K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

## Support Vector Machine
from sklearn.svm import SVC

## Neural Network
from sklearn.neural_network import MLPClassifier

## Boosting Methods
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Scoring
from sklearn.metrics import accuracy_score, f1_score
from sklearn.base import clone

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset

## Config


In [135]:
class Config:
    def __init__(self):
        self.seq_length = None
        self.input_size = None
        self.hidden_size = None
        self.num_layer = None
        self.batch_size = None

    def set_seed(self, seed):
        # Python
        random.seed(seed)
        # Numpy
        np.random.seed(seed)
        # Torch(CPU)
        torch.manual_seed(seed)
        # Torch(GPU)
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
            torch.cuda.manual_seed_all(seed)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

    def device(self):
        if torch.cuda.is_available():
            device = torch.device("cuda")
        else:
            device = torch.device("cpu")
        print(device)

    def set_path(self, path):
        self.path = path
        print(f"path setting : {self.path}")

config = Config()

# 시드 고정
config.set_seed(42)
config.device()

# path 고정
os.chdir('/content/drive/MyDrive/toy_project') # 변경바람
path_checkpoint = "checkpoints"
config.set_path("./data/")
os.makedirs(path_checkpoint, exist_ok = True)

cuda
path setting : ./data/


# Data

In [136]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [137]:
path = config.path
df = pd.read_csv(path + "Hotel_Reservations.csv")
temp = df.copy()
temp = temp.set_index("Booking_ID")
temp

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,Meal Plan 1,0,Room_Type 4,85,2018,8,3,Online,0,0,0,167.80,1,Not_Canceled
INN36272,2,0,1,3,Meal Plan 1,0,Room_Type 1,228,2018,10,17,Online,0,0,0,90.95,2,Canceled
INN36273,2,0,2,6,Meal Plan 1,0,Room_Type 1,148,2018,7,1,Online,0,0,0,98.39,2,Not_Canceled


## Feature Select

In [138]:
hotel = temp[['no_of_adults',
             'no_of_children',
             'no_of_weekend_nights',
             'no_of_week_nights',
             'type_of_meal_plan',
             'required_car_parking_space',
             'room_type_reserved',
             'lead_time',
             'arrival_year',
             'arrival_month',
             'arrival_date',
             'market_segment_type',
             'repeated_guest',
             'no_of_previous_cancellations',
             'no_of_previous_bookings_not_canceled',
             'avg_price_per_room',
             'no_of_special_requests',
             'booking_status']]

# EDA&PreProcessing

In [139]:
hotel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36275 entries, INN00001 to INN36275
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          36275 non-null  int64  
 1   no_of_children                        36275 non-null  int64  
 2   no_of_weekend_nights                  36275 non-null  int64  
 3   no_of_week_nights                     36275 non-null  int64  
 4   type_of_meal_plan                     36275 non-null  object 
 5   required_car_parking_space            36275 non-null  int64  
 6   room_type_reserved                    36275 non-null  object 
 7   lead_time                             36275 non-null  int64  
 8   arrival_year                          36275 non-null  int64  
 9   arrival_month                         36275 non-null  int64  
 10  arrival_date                          36275 non-null  int64  
 11  market_seg

In [140]:
# describe - numeric
hotel.describe()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests
count,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000,36275.000000
mean,1.844962,0.105279,0.810724,2.204300,0.030986,85.232557,2017.820427,7.423653,15.596995,0.025637,0.023349,0.153411,103.423539,0.619655
std,0.518715,0.402648,0.870644,1.410905,0.173281,85.930817,0.383836,3.069894,8.740447,0.158053,0.368331,1.754171,35.089424,0.786236
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,0.000000,17.000000,2018.000000,5.000000,8.000000,0.000000,0.000000,0.000000,80.300000,0.000000
50%,2.000000,0.000000,1.000000,2.000000,0.000000,57.000000,2018.000000,8.000000,16.000000,0.000000,0.000000,0.000000,99.450000,0.000000
75%,2.000000,0.000000,2.000000,3.000000,0.000000,126.000000,2018.000000,10.000000,23.000000,0.000000,0.000000,0.000000,120.000000,1.000000
max,4.000000,10.000000,7.000000,17.000000,1.000000,443.000000,2018.000000,12.000000,31.000000,1.000000,13.000000,58.000000,540.000000,5.000000


In [141]:
# descirbe - object
hotel.describe(include = "O")

,type_of_meal_plan,room_type_reserved,market_segment_type,booking_status
count,36275,36275,36275,36275
unique,4,7,5,2
top,Meal Plan 1,Room_Type 1,Online,Not_Canceled
freq,27835,28130,23214,24390


## Missing Value

In [142]:
# isnull?
hotel.isna().sum()

,0
no_of_adults,0
no_of_children,0
no_of_weekend_nights,0
no_of_week_nights,0
type_of_meal_plan,0
required_car_parking_space,0
room_type_reserved,0
lead_time,0
arrival_year,0
arrival_month,0


In [143]:
# # visualizaiton
# num_subplot = len(hotel.columns)
ls_columns = list(hotel.columns)

# fig, axes = plt.subplots(nrows = (num_subplot // 4) + 1, ncols = 4, figsize = (16, 12))
# plt.tight_layout(w_pad = 5, h_pad = 8)
# axes = axes.flatten()

# for i, col in enumerate(ls_columns):
#     if hotel[col].dtype == "object":
#         hotel[col].value_counts().plot(kind = "bar", ax = axes[i], color = "bisque")
#         axes[i].set_title(col)
#         axes[i].set_ylabel("count")
#         axes[i].tick_params(axis = 'x', rotation = 45)

#     else:
#         hotel[col].plot(kind = "hist", ax = axes[i], bins = 100, color = "paleturquoise")
#         axes[i].set_title(col)
#         axes[i].tick_params(axis = 'x', rotation = 45)

avg_price_per_room이 0이 나올 수 없음   
imputer를 사용하기 위해서 인코딩 이 후 결측치를 처리하도록 하겠음

## Encoding

In [144]:
# Label Encoding만 진행 - 원핫인코딩은 이미 되어있다고 판단 ∵ hotel.inof의 dtype확인시 범주형자료들 중 일부가 int64로 되어있음
encoder = LabelEncoder()

# encoding
for col in ls_columns:
    if hotel[col].dtype == "object":

        hotel[col + "_encode"] = encoder.fit_transform(hotel[col])
        hotel.drop(columns = [col], inplace = True)
hotel

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,type_of_meal_plan_encode,room_type_reserved_encode,market_segment_type_encode,booking_status_encode
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,0,224,2017,10,2,0,0,0,65.00,0,0,0,3,1
INN00002,2,0,2,3,0,5,2018,11,6,0,0,0,106.68,1,3,0,4,1
INN00003,1,0,2,1,0,1,2018,2,28,0,0,0,60.00,0,0,0,4,0
INN00004,2,0,0,2,0,211,2018,5,20,0,0,0,100.00,0,0,0,4,0
INN00005,2,0,1,1,0,48,2018,4,11,0,0,0,94.50,0,3,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,0,85,2018,8,3,0,0,0,167.80,1,0,3,4,1
INN36272,2,0,1,3,0,228,2018,10,17,0,0,0,90.95,2,0,0,4,0
INN36273,2,0,2,6,0,148,2018,7,1,0,0,0,98.39,2,0,0,4,1


## MissingValue - imputer

In [145]:
# ls_imputer
ls_imputer = [KNNImputer(), IterativeImputer()]

In [146]:
# set imputer - 변경바람
imputer = ls_imputer[0] # KNNImputer

In [147]:
# imputing
hotel["avg_price_per_room"].replace(0, np.nan, inplace = True) # 0 -> null

hotel["avg_price_per_room_impute"] = imputer.fit_transform(hotel[["avg_price_per_room"]])
hotel.drop(columns = ["avg_price_per_room"], inplace = True)
hotel

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,type_of_meal_plan_encode,room_type_reserved_encode,market_segment_type_encode,booking_status_encode,avg_price_per_room_impute
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,0,224,2017,10,2,0,0,0,0,0,0,3,1,65.00
INN00002,2,0,2,3,0,5,2018,11,6,0,0,0,1,3,0,4,1,106.68
INN00003,1,0,2,1,0,1,2018,2,28,0,0,0,0,0,0,4,0,60.00
INN00004,2,0,0,2,0,211,2018,5,20,0,0,0,0,0,0,4,0,100.00
INN00005,2,0,1,1,0,48,2018,4,11,0,0,0,0,3,0,4,0,94.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INN36271,3,0,2,6,0,85,2018,8,3,0,0,0,1,0,3,4,1,167.80
INN36272,2,0,1,3,0,228,2018,10,17,0,0,0,2,0,0,4,0,90.95
INN36273,2,0,2,6,0,148,2018,7,1,0,0,0,2,0,0,4,1,98.39


## Outlier - 많이 삭제되서 그대로 두겠음 (코드 수정필요)

In [148]:
# # define IQR
# def IQR(df):

#     exclude_cols = ['Booking_ID', 'type_of_meal_plan', 'room_type_reserved', 'market_segment_type', 'booking_status']
#     for col in hotel.columns:
#         if col not in exclude_cols and pd.api.types.is_numeric_dtype(df[col]):
#             Q1 = df[col].quantile(0.25)
#             Q3 = df[col].quantile(0.75)
#             IQR = Q3 - Q1

#             lower_bound = Q1 - 1.5 * IQR
#             upper_bound = Q3 + 1.5 * IQR

#             outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
#             df[col] = df[col][~outliers]  # 이상치가 아닌 값만 남김

#     df.dropna(inplace=True)
#     return df

# def z_score(df):
#     # 제외할 범주형 변수 리스트 설정
#     exclude_cols = ['Booking_ID', 'type_of_meal_plan', 'room_type_reserved',
#                     'market_segment_type', 'booking_status']

#     # 수치형 열 중 제외할 열을 필터링하여 처리할 열 선택
#     numeric_cols = [col for col in df.select_dtypes(include=[np.number]).columns if col not in exclude_cols]

#     # 수치형 열에 대해 Z-score를 계산
#     z_scores = df[numeric_cols].apply(zscore)  # 각 열에 Z-score 적용
#     abs_z_scores = np.abs(z_scores)  # Z-score의 절대값 계산

#     # Z-score가 3 이상인 경우 이상치로 간주하여 해당 행을 제거
#     df = df[(abs_z_scores < 3).all(axis=1)]  # 모든 열에서 Z-score가 3 미만인 행만 남김

#     return df

In [149]:
# # list of outlier remover
# ls_remover = [z_score, IQR]

In [150]:
# # set outlier remover
# remover = ls_remover[0] # zscore

In [151]:
# # remover outlier
# print(f"before size = {hotel.shape}")

# hotel = remover(hotel)

# print(f"after size = {hotel.shape}")

## Scaling

In [152]:
# list of scaler
ls_scaler = [StandardScaler(), MinMaxScaler(), RobustScaler()]

In [153]:
# set scaler - 변경 바람
scaler = ls_scaler[2] # RobustScaler : 이상치에 덜 민감

In [154]:
# scaling
hotel_scaled = scaler.fit_transform(hotel)
hotel_scaled = pd.DataFrame(columns = hotel.columns, data = hotel_scaled)
hotel_scaled

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,required_car_parking_space,lead_time,arrival_year,arrival_month,arrival_date,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,no_of_special_requests,type_of_meal_plan_encode,room_type_reserved_encode,market_segment_type_encode,booking_status_encode,avg_price_per_room_impute
0,0.0,0.0,0.0,0.0,0.0,1.532110,-1.0,0.4,-0.933333,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-0.897436
1,0.0,0.0,0.5,0.5,0.0,-0.477064,0.0,0.6,-0.666667,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.171282
2,-1.0,0.0,0.5,-0.5,0.0,-0.513761,0.0,-1.2,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.025641
3,0.0,0.0,-0.5,0.0,0.0,1.412844,0.0,-0.6,0.266667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.000000
4,0.0,0.0,0.0,-0.5,0.0,-0.082569,0.0,-0.8,-0.333333,0.0,0.0,0.0,0.0,3.0,0.0,0.0,-1.0,-0.141026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36270,1.0,0.0,0.5,2.0,0.0,0.256881,0.0,0.0,-0.866667,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,1.738462
36271,0.0,0.0,0.0,0.5,0.0,1.568807,0.0,0.4,0.066667,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-0.232051
36272,0.0,0.0,0.5,2.0,0.0,0.834862,0.0,-0.2,-1.000000,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,-0.041282
36273,0.0,0.0,-0.5,0.5,0.0,0.055046,0.0,-0.8,0.333333,0.0,0.0,0.0,0.0,3.0,0.0,0.0,-1.0,-0.141026


## Data Split

In [155]:
# define x and y,
# y : target
y = hotel_scaled["room_type_reserved_encode"] # target
x = hotel_scaled.drop(columns = ["room_type_reserved_encode"])

In [156]:
# data split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train.columns

Index(['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'required_car_parking_space', 'lead_time',
       'arrival_year', 'arrival_month', 'arrival_date', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'no_of_special_requests', 'type_of_meal_plan_encode',
       'market_segment_type_encode', 'booking_status_encode',
       'avg_price_per_room_impute'],
      dtype='object')

In [157]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29020 entries, 25629 to 15795
Data columns (total 17 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   no_of_adults                          29020 non-null  float64
 1   no_of_children                        29020 non-null  float64
 2   no_of_weekend_nights                  29020 non-null  float64
 3   no_of_week_nights                     29020 non-null  float64
 4   required_car_parking_space            29020 non-null  float64
 5   lead_time                             29020 non-null  float64
 6   arrival_year                          29020 non-null  float64
 7   arrival_month                         29020 non-null  float64
 8   arrival_date                          29020 non-null  float64
 9   repeated_guest                        29020 non-null  float64
 10  no_of_previous_cancellations          29020 non-null  float64
 11  no_of_previous_b

In [158]:
# k-fold
ls_kfold = [KFold, StratifiedKFold]

kfold = ls_kfold[1] # k-fold
cv = kfold(n_splits = 5, shuffle = True, random_state = 42)

# Modeling - ML

## Model list

In [159]:
# ML = [
#     # Ensemble Methods
#     AdaBoostClassifier(),
#     BaggingClassifier(),
#     ExtraTreesClassifier(),
#     GradientBoostingClassifier(),
#     RandomForestClassifier(),

#     # GLM
#     LogisticRegression(),
#     PassiveAggressiveClassifier(),

#     # Trees
#     DecisionTreeClassifier(),
#     ExtraTreeClassifier(),

#     # XGBoost
#     XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor')]

In [160]:
ML = [
    # Ensemble Methods
    RandomForestClassifier(),
    AdaBoostClassifier(),
    BaggingClassifier(),
    GradientBoostingClassifier(),

    # GLM
    LogisticRegression(),

    # Trees
    DecisionTreeClassifier(),

    # Naive Bayes
    GaussianNB(),

    # K-Nearest Neighbors
    KNeighborsClassifier(),

    # Support Vector Machine
    SVC(),

    # Boosting Methods
    XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor'),
    LGBMClassifier(),
    CatBoostClassifier(verbose=0)
]

## hyper_param 4 ml

In [161]:
# 하이퍼파라미터 조정
# param_distributions_map = {
#     "AdaBoostClassifier": {
#         "n_estimators": [3, 5, 7, 9],  # 사용할 약한 학습기(기본적으로 DecisionTree)의 개수
#         "learning_rate": [0.01, 0.1, 0.2]  # 학습 속도를 조절하는 하이퍼파라미터, 값이 낮을수록 느리지만 안정적임
#     },
#     "BaggingClassifier": {
#         "n_estimators": [10, 50, 100],  # 개별 학습기(base estimator)의 개수
#         "max_samples": [0.5, 0.7, 1.0],  # 각 학습기에 사용할 샘플의 비율
#         "max_features": [0.5, 0.7, 1.0],  # 각 학습기에 사용할 피처의 비율
#         "bootstrap": [True],  # 학습 데이터를 랜덤하게 복원 추출할지 여부
#         "oob_score": [True, False]  # Out-of-Bag 샘플을 사용하여 모델 평가를 할지 여부
#     },
#     "ExtraTreesClassifier": {
#         "n_estimators": [50, 100, 200],  # 트리의 개수
#         "max_depth": [None, 10, 20],  # 트리의 최대 깊이, None이면 리프 노드까지 확장
#         "min_samples_split": [2, 5]  # 노드를 분할하기 위한 최소 샘플 수
#     },
#     "GradientBoostingClassifier": {
#         "n_estimators": [50, 100],  # 사용할 약한 학습기(트리)의 개수
#         "learning_rate": [0.01, 0.1],  # 각 단계에서 학습 속도를 조절하는 파라미터
#         "max_depth": [3, 5],  # 개별 트리의 최대 깊이
#         "loss": ["log_loss"]  # 손실 함수 종류, 분류에서는 로그 손실(log loss) 사용
#     },
#     "RandomForestClassifier": {
#         "n_estimators": [50, 100],  # 트리의 개수
#         "max_depth": [None, 10],  # 트리의 최대 깊이, None이면 리프 노드까지 확장
#         "min_samples_split": [2, 5],  # 노드를 분할하기 위한 최소 샘플 수
#         "bootstrap": [True],  # 학습 데이터를 랜덤하게 복원 추출할지 여부
#         "oob_score": [True, False]  # Out-of-Bag 샘플을 사용하여 모델 평가를 할지 여부
#     },
#     "LogisticRegression": {
#         "Cs": [1, 10],  # 규제 강도를 결정하는 파라미터, 값이 클수록 규제가 약함
#         "max_iter": [100, 200]  # 최대 반복 횟수
#     },
#     "PassiveAggressiveClassifier": {
#         "C": [0.01, 0.1, 1],  # 규제 강도를 결정하는 파라미터, 값이 클수록 규제가 약함
#         "max_iter": [1000, 2000]  # 최대 반복 횟수
#     },
#     "DecisionTreeClassifier": {
#         "max_depth": [None, 10, 20],  # 트리의 최대 깊이, None이면 리프 노드까지 확장
#         "min_samples_split": [2, 5]  # 노드를 분할하기 위한 최소 샘플 수
#     },
#     "ExtraTreeClassifier": {
#         "max_depth": [None, 10, 20],  # 트리의 최대 깊이, None이면 리프 노드까지 확장
#         "min_samples_split": [2, 5]  # 노드를 분할하기 위한 최소 샘플 수
#     },
#     "XGBClassifier": {
#         "n_estimators": [50, 100],  # 사용할 약한 학습기(트리)의 개수
#         "learning_rate": [0.01, 0.1],  # 각 단계에서 학습 속도를 조절하는 파라미터
#         "max_depth": [3, 5]  # 개별 트리의 최대 깊이
#     }
# }


In [165]:
param_distributions_map = {
    # Ensemble Methods
    "RandomForestClassifier": {
        "n_estimators": [50, 100, 200],  # 트리의 개수
        "max_depth": [None, 10, 20],  # 트리의 최대 깊이
        "min_samples_split": [2, 5],  # 노드를 분할하기 위한 최소 샘플 수
        "bootstrap": [True],  # 부트스트랩 샘플링 사용 여부
        "max_features": ["sqrt", "log2", None]  # 각 트리에 사용할 피처 수
    },
    "AdaBoostClassifier": {
        "n_estimators": [50, 100, 200],  # 약한 학습기(기본적으로 DecisionTree)의 개수
        "learning_rate": [0.01, 0.1, 1]  # 학습 속도
    },
    "BaggingClassifier": {
        "n_estimators": [10, 50, 100],  # 개별 학습기의 개수
        "max_samples": [0.5, 0.7, 1.0],  # 각 학습기에 사용할 샘플의 비율
        "max_features": [0.5, 0.7, 1.0],  # 각 학습기에 사용할 피처의 비율
        "bootstrap": [True],  # 부트스트랩 샘플링 사용 여부
        "oob_score": [True, False]  # OOB 샘플을 사용하여 평가할지 여부
    },
    "GradientBoostingClassifier": {
        "n_estimators": [50, 100, 200],  # 약한 학습기(트리)의 개수
        "learning_rate": [0.01, 0.1, 0.2],  # 학습 속도
        "max_depth": [3, 5, 7]  # 트리의 최대 깊이
    },

    # GLM
    "LogisticRegression": {
        "C": [0.1, 1, 10],  # 규제 강도
        "max_iter": [100, 200, 500]  # 최대 반복 횟수
    },

    # Trees
    "DecisionTreeClassifier": {
        "max_depth": [None, 10, 20],  # 트리의 최대 깊이
        "min_samples_split": [2, 5, 10]  # 노드를 분할하기 위한 최소 샘플 수
    },

    # Naive Bayes
    "GaussianNB": {
        "var_smoothing": [1e-9, 1e-8, 1e-7]  # 가우시안 분포의 분산에 추가할 작은 수
    },

    # K-Nearest Neighbors
    "KNeighborsClassifier": {
        "n_neighbors": [3, 5, 7, 9],  # 이웃의 개수
        "weights": ["uniform", "distance"],  # 가중치 방식
        "p": [1, 2]  # 거리 척도 (1: 맨해튼 거리, 2: 유클리드 거리)
    },

    # Support Vector Machine
    "SVC": {
        "C": [0.1, 1, 10],  # 규제 강도
        "kernel": ["linear", "rbf", "poly"],  # 커널 유형
        "gamma": ["scale", "auto"]  # 커널 계수
    },

    # Boosting Methods
    "XGBClassifier": {
        "n_estimators": [50, 100, 200],  # 약한 학습기(트리)의 개수
        "learning_rate": [0.01, 0.1, 0.2],  # 학습 속도
        "max_depth": [3, 5, 7],  # 트리의 최대 깊이
        "subsample": [0.7, 0.8, 1.0],  # 각 트리에 사용할 샘플의 비율
        "colsample_bytree": [0.7, 0.8, 1.0]  # 각 트리에 사용할 피처의 비율
    },
    "LGBMClassifier": {
        "n_estimators": [50, 100, 200],  # 약한 학습기(트리)의 개수
        "learning_rate": [0.01, 0.1, 0.2],  # 학습 속도
        "max_depth": [-1, 10, 20],  # 트리의 최대 깊이
        "num_leaves": [31, 50, 100]  # 트리의 리프 노드 개수
    },
    "CatBoostClassifier": {
        "iterations": [50, 100, 200],  # 학습 반복 횟수
        "learning_rate": [0.01, 0.1, 0.2],  # 학습 속도
        "depth": [3, 5, 7],  # 트리의 깊이
        "l2_leaf_reg": [1, 3, 5]  # L2 정규화 계수
    }
}


## Train

In [166]:
results = {} # 모델 결과 저장

best_model_overall = None
best_accuracy_overall = 0
best_model_name = ""
best_epoch = 0

In [ ]:
for model in tqdm(ML, desc="Model Training"):
    model_name = model.__class__.__name__
    print(f"\nProcessing {model_name}...")

    param_distributions = param_distributions_map.get(model_name, {})  # 파라미터 가져옴
    model_checkpoint_dir = os.path.join(path_checkpoint, model_name)  # model_checkpoint_dir
    os.makedirs(model_checkpoint_dir, exist_ok=True)

    best_epoch_accuracy = 0
    best_epoch_f1 = 0
    best_epoch_params = {}
    best_epoch_model = None

    # Randomized Search
    for epoch in tqdm(range(1, 11), desc=f"Training {model_name}"):  # tqdm 추가 (1, 11은 테스트, 실제는 1, 101 ~ 1001)
        random_search = RandomizedSearchCV(
            estimator=clone(model),
            param_distributions=param_distributions,  # 하이퍼파라미터
            n_iter=10,  # 하이퍼파라미터 조합 수
            scoring="accuracy",
            n_jobs=-1,
            cv=cv,
            random_state=42 + epoch  # 각 에포크마다 시드가 다르게 설정
        )

        # train
        random_search.fit(x_train, y_train)

        # predict
        best_model = random_search.best_estimator_
        y_pred_test = best_model.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred_test)
        f1 = f1_score(y_test, y_pred_test, average='weighted')  # F1-score 계산, 다중 클래스 시 weighted 사용

        # save checkpoint
        checkpoint_path = os.path.join(model_checkpoint_dir, f"{model_name}_epoch_{epoch}.pkl")
        joblib.dump(best_model, checkpoint_path)

        # update
        if accuracy > best_epoch_accuracy or (accuracy == best_epoch_accuracy and f1 > best_epoch_f1):
            best_epoch_accuracy = accuracy
            best_epoch_f1 = f1
            best_epoch_params = random_search.best_params_
            best_epoch_model = best_model
            best_epoch = epoch

        print(f"Epoch {epoch} - Validation Accuracy for {model_name}: {accuracy:.4f}, F1 Score: {f1:.4f}")

    # 모델별 최고 성능 저장
    results[model_name] = {
        "Best Params": best_epoch_params,
        "Best Estimator": best_epoch_model,
        "Validation Accuracy": best_epoch_accuracy,
        "Validation F1 Score": best_epoch_f1,
        "Best Epoch": best_epoch
    }

    print(f"Best Params for {model_name}: {best_epoch_params}")
    print(f"Validation Accuracy for {model_name}: {best_epoch_accuracy:.4f}, F1 Score: {best_epoch_f1:.4f} at Epoch {best_epoch}")

    # 전체 모델 중 최고 성능 모델 갱신
    if best_epoch_accuracy > best_accuracy_overall or (best_epoch_accuracy == best_accuracy_overall and best_epoch_f1 > best_epoch_model.best_f1):
        best_accuracy_overall = best_epoch_accuracy
        best_model_overall = best_epoch_model
        best_model_name = model_name

# 최종 추천 모델 출력
print(f"\nRecommended Model: {best_model_name} with Accuracy: {best_accuracy_overall:.4f}")
print(f"Best Epoch for {best_model_name}: {best_epoch}")

# K-fold cross-validation을 사용하여 전체 성능 평가
final_kfold_results_acc = cross_val_score(best_model_overall, x_train, y_train, cv=cv, scoring='accuracy')
final_kfold_results_f1 = cross_val_score(best_model_overall, x_train, y_train, cv=cv, scoring='f1_weighted')
print(f"K-Fold Cross Validation Accuracy Results: {final_kfold_results_acc}")
print(f"Mean Accuracy: {final_kfold_results_acc.mean():.4f}")
print(f"K-Fold Cross Validation F1 Results: {final_kfold_results_f1}")
print(f"Mean F1 Score: {final_kfold_results_f1.mean():.4f}")


Model Training:   0%|          | 0/12 [00:00<?, ?it/s]


Processing RandomForestClassifier...



Training RandomForestClassifier:  10%|█         | 1/10 [00:52<07:50, 52.31s/it]

Epoch 1 - Validation Accuracy for RandomForestClassifier: 0.9227, F1 Score: 0.9183



Training RandomForestClassifier:  20%|██        | 2/10 [01:55<07:51, 58.89s/it]

...

Epoch 3 - Validation Accuracy for RandomForestClassifier: 0.9225, F1 Score: 0.9176



Training RandomForestClassifier:  40%|████      | 4/10 [03:55<05:50, 58.44s/it]

Epoch 4 - Validation Accuracy for RandomForestClassifier: 0.9242, F1 Score: 0.9199



Training RandomForestClassifier:  50%|█████     | 5/10 [05:10<05:22, 64.46s/it]

Epoch 5 - Validation Accuracy for RandomForestClassifier: 0.9249, F1 Score: 0.9205


# Modeling - DL